## 4.4.2 Embeddingレイヤの実装 

In [1]:
import numpy as np

In [2]:
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [4]:
W[5]

array([15, 16, 17])

In [5]:
idx = np.array([1, 0, 1, 3])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 3,  4,  5],
       [ 9, 10, 11]])

In [8]:
class Embedding(object):
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        #dW[self.idx] = dout # 実は悪い例
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return 

In [9]:
class EmbeddingDot(object):
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, taget_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dtou.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [12]:
import sys
sys.path.append('..')
from common.layers import SigmoidWithLoss

In [14]:
class NegativeSamplingLoss(object):
    """
    Parameters:
    ------------------------
    W: 出力側の重み
    corpus: 単語IDのリスト
    power: 確率分布の累乗の値
    sample_size: 負例のサンプリング数
    """
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        # 正例用1 + 負例sample_size分用意。リストの最初が正例を扱うものとする。
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        # 正例用1 + 負例sample_size分用意。リストの最初が正例を扱うものとする。
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 正例のforward
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 負例のforward
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negetive_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negetive_label)
            
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        return dh

# 4.3 改良版word2vecの学習 

In [16]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        """
        Attribute:
        -------------------
        vocab_size: 語彙数
        hidden_size: 中間層のニューロン数
        window_size: コンテキストのサイズ
        corpus: 単語IDのリスト
        """
        V, H = vocab_size, hidden_size
        
        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # 全ての重みと勾配を配列にまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return 

## 4.3.2 CBOWモデルの学習コード

In [18]:
import sys
sys.path.append('..')
import numpy as np
from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# ハイパーパラメータの調整
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)

# モデルなどの生成
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 学習開始
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 後ほど利用できるように、必要なデータを保存
word_vecs = model.word_vecs
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| epoch 1 |  iter 1 / 9295 | time 0[s] | loss 4.16
| epoch 1 |  iter 21 / 9295 | time 1[s] | loss 4.16
| epoch 1 |  iter 41 / 9295 | time 2[s] | loss 4.15
| epoch 1 |  iter 61 / 9295 | time 3[s] | loss 4.12
| epoch 1 |  iter 81 / 9295 | time 4[s] | loss 4.05
| epoch 1 |  iter 101 / 9295 | time 5[s] | loss 3.92
| epoch 1 |  iter 121 / 9295 | time 6[s] | loss 3.78
| epoch 1 |  iter 141 / 9295 | time 7[s] | loss 3.63
| epoch 1 |  iter 161 / 9295 | time 8[s] | loss 3.50
| epoch 1 |  iter 181 / 9295 | time 10[s] | loss 3.36
| epoch 1 |  iter 201 / 9295 | time 11[s] | loss 3.25
| epoch 1 |  iter 221 / 9295 | time 12[s] | loss 3.16
| epoch 1 |  iter 241 / 9295 | time 13[s] | loss 3.08
| epoch 1 |  iter 261 / 9295 | time 14[s] | loss 3.02
| epoch 1 |  iter 281 / 9295 | time 16[s] | loss 2.95
| epoch 1 |  iter 301 / 9295 | time 17[s] | loss 2.90
| epoch 1 |  iter 321 / 9295 | time 18[s] | loss 2.86
| epoch 1 |  iter 341 / 9295 | time 20[s] | loss 2.86
| epoch 1 |  iter 361 / 9295 | time 21[s] |

KeyboardInterrupt: 

## 4.3.3 CBOWモデルの評価

In [19]:
import sys
sys.path.append('..')
from common.util import most_similar
import pickle

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']
    
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
